# The PolInSAR Course - May 6, 2024
# SAR Interferometry (InSAR) 
# Part 2: The InSAR Coherence & Height Inversion

* TanDEM-X acquisition over Mondah forest (Gabon)
* Path: 'data/insar/'
* SLCs: master_HH_Mondah_S_2015_11_11_cut.rat, slave_HH_Mondah_S_2015_11_11_cut.rat
* Flat-earth: flat_earth_Mondah_S_2015_11_11_cut.rat
* Full-waveform Lidar (LVIS) RH100: RH100_Mondah_S_2015_11_11_cut.rat

Objective:
- Calculate InSAR coherence with a 20 m resolution, estimate forest height using the provided formulas, and validate the result against lidar

Tips:
- Compensate system decorrelation 0.97
- Invalid points in the RH100 are set to -9999

In [1]:
# --- Download exercise data & import reader function
from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/insar')

# --- Import useful libraries, functions, and modules
import sys
sys.path.append('/projects/src/')
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from numpy.fft import fft2, ifft2

%matplotlib widget


/projects/src/ste_io.py:69: UserWarning: Failed to import STEtools.visualisation. write_pixmap will not work.
  warnings.warn("Failed to import STEtools.visualisation. write_pixmap will not work.")


In [2]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    corr = uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    return corr

**Input parameters**

In [3]:
# path 2 images
path = 'data/insar/'

# Input TDX pixel spacing, in meters
spacrg = 1.36411
spacaz = 1.86817

# TDX vertical wavenumber, in rad / meters
kz = 0.06092195

# Output range resolution, in meters
resrg = 20.
resaz = 20.

**Step 1: Open images, and visualize**

In [5]:
# --- open images
im1 = rrat(path + 'master_HH_Mondah_S_2015_11_11_cut.rat')
im2 = rrat(path + 'slave_HH_Mondah_S_2015_11_11_cut.rat')

# --- display
amp1 = np.sqrt( np.abs( calculate_covariance(im1, im1, 7, 7) ) )
plt.figure(figsize = (10, 5))
plt.imshow(amp1, vmin = 0, vmax = 2*np.mean(amp1), aspect = 'auto', cmap = 'gray' )
plt.title('Amplitude Master')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 2 : Compensate flat-earth**

In [6]:
# --- open flat-earth phase
fe = rrat(path + 'flat_earth_Mondah_S_2015_11_11_cut.rat')

# --- compensate
im2_fe = im2 * np.exp(1j * fe)


**Step 3 : Calculate coherence**

In [12]:
# --- calculate number of looks
looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1

# --- calculate coherence and display
coherence = calculate_covariance(im1, im2_fe, looksr, looksa)
amp1 = np.sqrt( np.abs(calculate_covariance(im1, im1, looksr, looksa)) )
amp2 = np.sqrt( np.abs(calculate_covariance(im2_fe, im2_fe, looksr, looksa)) )

coherence = coherence / amp1 / amp2

# --- display 
plt.figure(figsize = (10, 10))
plt.subplot(2, 1, 1)
plt.imshow(np.abs(coherence), vmin = 0, vmax = 1, aspect = 'auto', cmap = 'gray' )
plt.title('Coherence absolute value')
plt.colorbar()
plt.subplot(2, 1, 2)
plt.imshow(np.angle(coherence), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet' )
plt.title('Coherence phase')
plt.colorbar()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 4 : Invert for forest height (closed-form)**

In [11]:
# --- Compensate system decorrelation
gammavol = np.abs(coherence) / 0.97
gammavol[ gammavol > 1] = 1

In [13]:
# --- height inversion
hv = 2*np.pi / kz * (1 - 2/np.pi*np.arcsin(gammavol**0.8) )

# --- display
plt.figure(figsize = (10, 10))
plt.subplot(2, 1, 1)
plt.imshow(hv, vmin = 0, vmax = 60, cmap = 'jet' )
plt.title('Inverted forest height')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Step 5 : Validation**

In [ ]:
# --- open lidar reference height
rh100 = rrat(path + 'RH100_Mondah_S_2015_11_11_cut.rat')

# --- display
plt.figure(figsize = (10, 10))
plt.subplot(2, 1, 1)
plt.imshow(hv, vmin = 0, vmax = 60, aspect = 'auto', cmap = 'jet' )
plt.title('Inverted forest height')
plt.colorbar()
plt.subplot(2, 1, 2)
plt.imshow(rh100, vmin = 0, vmax = 60, aspect = 'auto', cmap = 'jet' )
plt.title('Lidar height RH100')
plt.colorbar()

In [ ]:
# --- validation histogram
ind = (rh100 != -9999)
hv_lim = (5,60)
hist2d_hv = np.histogram2d(rh100[ind], hv[ind], bins=120, range=[hv_lim, hv_lim])

# --- display histogram
plt.figure(figsize=(5,5))
plt.imshow( np.flipud(np.transpose(hist2d_hv[0])), cmap='jet', extent=[hv_lim[0], hv_lim[1],hv_lim[0], hv_lim[1]])
plt.plot(hv_lim, hv_lim, lw=2, color='w', linestyle='--')
plt.xlabel('Lidar height [m]')
plt.ylabel('TDX height [m]')
plt.grid()